In [1]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Load data

In [2]:
# Load data
csv_file = '../data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df = pd.read_csv(csv_file,parse_dates=[7])

# Datetime index
df = df.set_index('Date_time_R')

# Create u and v components with units m/s
df['u'] = df['Easting (cm/sec)']/100
df['v'] = df['Northing (cm/sec)']/100

### Start and end times for calculations

Show start and end times for each deployment

Time period of interest: 14 October 2014 to 13 Nov 2017

In [3]:
t1_list = [np.datetime64('2014-10-16'),
           np.datetime64('2015-11-10'),
           np.datetime64('2016-11-14'),
           np.datetime64('2017-11-13')]

t2_list = [np.datetime64('2015-06-17'),
           np.datetime64('2016-11-08'),
           np.datetime64('2017-11-09'),
           np.datetime64('2018-10-17')]

plt.figure(figsize=(7.5,8))
plt.subplot(211)
plt.plot(df.index,df['u'])
# plt.xlim([t1_list[0]-np.timedelta64(3,'D'),
#           t2_list[-1]+np.timedelta64(3,'D')])
yl = plt.ylim()

for t1,t2 in zip(t1_list,t2_list):
    plt.plot([t1,t1],yl,'g')
    plt.plot([t2,t2],yl,'r')
    
plt.ylim(yl)
plt.title('eastward velocity')
plt.ylabel('[m/s]')
plt.axhline(0,linestyle='--',color='k')

plt.subplot(212)
plt.plot(df.index,df['v'])
# plt.xlim([t1_list[0]-np.timedelta64(3,'D'),
#           t2_list[-1]+np.timedelta64(3,'D')])
yl = plt.ylim()

for t1,t2 in zip(t1_list,t2_list):
    plt.plot([t1,t1],yl,'g')
    plt.plot([t2,t2],yl,'r')
    
plt.ylim(yl)
plt.title('northward velocity')
plt.ylabel('[m/s]')
plt.axhline(0,linestyle='--',color='k')

FigureCanvasNbAgg()

### Progressive vector diagrams

For each deployment, two plots are shown:
* A simple trajectory plot, with the starting point
* A trajectory plot colored by time

In [4]:
# time between each sample (in seconds)
dt = np.diff(df.index)/np.timedelta64(1,'s')

# unix time
df['unix_time'] =  (df.index - pd.Timestamp('1970-01-01 00:00:00'))/np.timedelta64(1,'s')

# loop through each time period
for t1,t2 in zip(t1_list,t2_list):
    
    # find data in time period
    ii, = np.where((df.index >= t1) & (df.index <= t2))
    
    u = np.array(df['u'])
    v = np.array(df['v'])
    
    fi, = np.where((np.isfinite(u[ii][:-1]+u[ii][1:])) & 
                   (np.isfinite(v[ii][:-1]+v[ii][1:])) &
                   (np.isfinite(dt[ii][:-1])))
    
    # cumulative integral using trapezoidal rule
    x = np.cumsum((0.5*(u[ii][:-1]+u[ii][1:])
                  *dt[ii][:-1])[fi],axis=0)
    y = np.cumsum((0.5*(v[ii][:-1]+v[ii][1:])
                  *dt[ii][:-1])[fi],axis=0)
    t = np.cumsum((dt[ii][:-1])[fi],axis=0)
           
    # plot progressive vector diagram
    plt.figure()
    plt.plot(x/1000,y/1000)
    plt.plot(0,0,'go')
    plt.gca().axis('equal')
    plt.xlabel('distance east [km]')
    plt.ylabel('distance north [km]')
    plt.title(str(t1)+' - '+str(t2))
    plt.savefig('figures/progressive_vector_original_'+
            str(t1.astype(object).year)+'-'+
            str(t2.astype(object).year)+
            '.pdf')
    
    plt.figure()
    plt.scatter(x/1000,y/1000,6,t,cmap='plasma')
    plt.gca().axis('equal')
    plt.xlabel('distance east [km]')
    plt.ylabel('distance north [km]')
    plt.title(str(t1)+' - '+str(t2))
    
    plt.figure()
    plt.hist2d(df['u'][ii],df['v'][ii],bins=50,cmap='gist_heat_r');
    plt.colorbar();
    plt.xlabel('eastward velocity [m/s]')
    plt.xlabel('northward velocity [m/s]')    
    plt.title(str(t1)+' - '+str(t2))
    xl = plt.xlim()
    yl = plt.ylim()
    plt.hlines(0,2*xl[0],2*xl[1])
    plt.vlines(0,2*yl[0],2*yl[1])
    plt.xlim(xl)
    plt.ylim(yl)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

...

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [5]:
for t1,t2 in zip(t1_list,t2_list):
    
    # find data in time period
    ii, = np.where((df.index >= t1) & (df.index <= t2))
    
    # cumulative integral using trapezoidal rule
    x = np.cumsum(0.5*(df['u'][ii][:-1]+df['u'][ii][1:])
                  *dt[ii],axis=0)
    y = np.cumsum(0.5*(df['v'][ii][:-1]+df['v'][ii][1:])
                  *dt[ii],axis=0)
    t = np.cumsum(dt[ii],axis=0)
           
    # plot progressive vector diagram
    plt.figure(figsize=(5,5))
    plt.plot(x/1000,y/1000,'k-')
    plt.plot(0,0,'o',color='gray')
    plt.gca().axis('equal')    
    plt.xlim([-140,50])
    plt.xticks(np.arange(-150,75,50))
    plt.ylim([-140,80])
    plt.yticks(np.arange(-150,75,50))
    plt.xlabel('distance east [km]')
    plt.ylabel('distance north [km]')
    plt.title(str(t1)+' - '+str(t2))
    plt.tight_layout()
    plt.savefig('figures/progressive_vector_'+
                str(t1.astype(object).year)+'-'+
                str(t2.astype(object).year)+
                '.pdf')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()